# Financial Analysis: Asset Stranding & Profitability Impact

## Purpose
This notebook provides **financial lens** on stranded asset risk, answering:
1. **When do assets become unprofitable?** (Break-even carbon prices)
2. **How many assets strand at each price point?** (Stranding thresholds)
3. **What % of revenue is consumed by carbon costs?** (Profitability impact)
4. **Which mine types/sectors are most vulnerable?** (Sector breakdown)
5. **Are emissions getting better or worse?** (2021-2024 trends)

**Key Metrics:**
- Break-even carbon price ($/tCO₂)
- Carbon cost as % of revenue
- Stranding cascade analysis
- Emission trajectory (improving vs. deteriorating)

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

# Styling
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

SCENARIO_COLORS = {
    '$50/tCO₂': '#2E86AB',
    '$100/tCO₂': '#A23B72',
    '$150/tCO₂': '#F18F01',
    '$200/tCO₂': '#C73E1D'
}

print('✓ Libraries loaded')

✓ Libraries loaded


## Data Loading & Financial Parameter Setup

In [20]:
# Load cleaned datasets
df = pd.read_csv('../datasets/output_data/cleaning/copper_mining_cleaned.csv')
df_with_risk = pd.read_csv('../datasets/output_data/stranded_assets/copper_mining_with_risk_categories.csv')

# Load monthly data for temporal analysis
df_monthly = pd.read_csv('../datasets/input_data/copper-mining_emissions_sources_v5_2_0.csv')
df_monthly['start_time'] = pd.to_datetime(df_monthly['start_time'])
df_monthly['year'] = df_monthly['start_time'].dt.year
df_monthly['month'] = df_monthly['start_time'].dt.month
df_monthly.rename(columns={'emissions_quantity': 'emissions_t_co2e'}, inplace=True)

print(f"Assets: {len(df):,}")
print(f"Production mines: {(df['mine_status']=='Production').sum():,}")
print(f"Monthly records: {len(df_monthly):,}")
print(f"Time range: {df_monthly['start_time'].min().date()} to {df_monthly['start_time'].max().date()}")

Assets: 914
Production mines: 608
Monthly records: 53,012
Time range: 2021-01-01 to 2025-10-01


In [21]:
# Copper market (2024 average)
COPPER_PRICE_PER_TONNE = 9500  # USD

# Mining economics
AVERAGE_ORE_GRADE = 0.008  # 0.8% Cu content (conservative industry average)
OPERATING_MARGIN = 0.30  # 30% typical operating margin for copper mining

# Carbon pricing scenarios
CARBON_PRICES = [0, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250]

print("Financial Parameters:")
print(f"  Copper price: ${COPPER_PRICE_PER_TONNE:,}/tonne")
print(f"  Ore grade assumption: {AVERAGE_ORE_GRADE*100}%")
print(f"  Operating margin: {OPERATING_MARGIN*100}%")
print(f"  Stranding threshold: Carbon cost > {OPERATING_MARGIN*100}% of revenue")

Financial Parameters:
  Copper price: $9,500/tonne
  Ore grade assumption: 0.8%
  Operating margin: 30.0%
  Stranding threshold: Carbon cost > 30.0% of revenue


---
## PART 1: Break-Even Analysis & Stranding Thresholds

### Methodology
- **Revenue** = Production × Ore Grade × Copper Price
- **Break-even carbon price** = (Revenue × Operating Margin) / Annual Emissions
- **Asset is "stranded"** when: Carbon Cost > Operating Margin

In [22]:
# Filter to production mines with calculable metrics
df_financial = df[
    (df['mine_status'] == 'Production') & 
    (df['annual_production_t_ore'] > 0) &
    (df['annual_emissions_t_co2e'] > 0)
].copy()

# Calculate revenue
df_financial['copper_produced_t'] = df_financial['annual_production_t_ore'] * AVERAGE_ORE_GRADE
df_financial['estimated_revenue_usd'] = df_financial['copper_produced_t'] * COPPER_PRICE_PER_TONNE

# Carbon cost as % of revenue for all scenarios
for scenario in [50, 100, 150, 200]:
    col_cost = f'carbon_cost_usd_{scenario}'
    col_pct = f'carbon_cost_pct_revenue_{scenario}'
    df_financial[col_pct] = (df_financial[col_cost] / df_financial['estimated_revenue_usd']) * 100

# BREAK-EVEN CARBON PRICE
df_financial['break_even_carbon_price'] = (
    (df_financial['estimated_revenue_usd'] * OPERATING_MARGIN) / 
    df_financial['annual_emissions_t_co2e']
).replace([np.inf, -np.inf], np.nan)

# Underwater flags (carbon cost exceeds operating margin)
for price in [25, 50, 75, 100, 125, 150, 175, 200, 250]:
    col_pct = f'carbon_cost_pct_revenue_{price}'
    if col_pct not in df_financial.columns:
        df_financial[col_pct] = (df_financial['annual_emissions_t_co2e'] * price / df_financial['estimated_revenue_usd']) * 100
    df_financial[f'underwater_at_{price}'] = df_financial[col_pct] > (OPERATING_MARGIN * 100)


print(f"financial metrics summary (n={len(df_financial)} production assets)")
print(f"Total estimated annual revenue: ${df_financial['estimated_revenue_usd'].sum()/1e9:.2f}B")
print(f"\nAssets 'underwater' (carbon cost > 30% margin):")
for price in [50, 100, 150, 200]:
    count = df_financial[f'underwater_at_{price}'].sum()
    pct = count / len(df_financial) * 100
    print(f"  At ${price:>3}/tCO₂: {count:>3} assets ({pct:>5.1f}%)")

# Break-even statistics
breakeven_valid = df_financial['break_even_carbon_price'].dropna()
breakeven_valid = breakeven_valid[(breakeven_valid > 0) & (breakeven_valid < 1000)]

print(f"\nBreak-even Carbon Price Statistics:")
print(f"  Median: ${breakeven_valid.median():.0f}/tCO₂")
print(f"  Mean: ${breakeven_valid.mean():.0f}/tCO₂")
print(f"  25th percentile: ${breakeven_valid.quantile(0.25):.0f}/tCO₂")
print(f"  75th percentile: ${breakeven_valid.quantile(0.75):.0f}/tCO₂")

financial metrics summary (n=601 production assets)
Total estimated annual revenue: $770.87B

Assets 'underwater' (carbon cost > 30% margin):
  At $ 50/tCO₂:   0 assets (  0.0%)
  At $100/tCO₂:   2 assets (  0.3%)
  At $150/tCO₂:   2 assets (  0.3%)
  At $200/tCO₂:   3 assets (  0.5%)

Break-even Carbon Price Statistics:
  Median: $776/tCO₂
  Mean: $706/tCO₂
  25th percentile: $551/tCO₂
  75th percentile: $938/tCO₂


#### 1.1 Stranding Cascade - How Many Assets at Each Price Point?

In [23]:
# Calculate stranding cascade
cascade_data = []
for price in range(0, 251, 5):  # Every $5 increment
    carbon_cost_pct = (df_financial['annual_emissions_t_co2e'] * price / df_financial['estimated_revenue_usd']) * 100
    stranded_count = (carbon_cost_pct > OPERATING_MARGIN * 100).sum()
    stranded_pct = stranded_count / len(df_financial) * 100
    
    cascade_data.append({
        'carbon_price': price,
        'stranded_assets': stranded_count,
        'stranded_pct': stranded_pct
    })

df_cascade = pd.DataFrame(cascade_data)

# Create waterfall visualization
fig1 = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Number of Stranded Assets by Carbon Price', 'Percentage of Portfolio Stranded'),
    row_heights=[0.5, 0.5],
    vertical_spacing=0.15
)

# Count plot
fig1.add_trace(
    go.Scatter(
        x=df_cascade['carbon_price'],
        y=df_cascade['stranded_assets'],
        mode='lines',
        name='Stranded Assets',
        fill='tozeroy',
        line=dict(color='#C73E1D', width=3),
        hovertemplate='Price: $%{x}/tCO₂<br>Stranded: %{y} assets<extra></extra>'
    ),
    row=1, col=1
)

# Percentage plot
fig1.add_trace(
    go.Scatter(
        x=df_cascade['carbon_price'],
        y=df_cascade['stranded_pct'],
        mode='lines',
        name='% Stranded',
        fill='tozeroy',
        line=dict(color='#A23B72', width=3),
        hovertemplate='Price: $%{x}/tCO₂<br>Stranded: %{y:.1f}%<extra></extra>'
    ),
    row=2, col=1
)

# Add scenario reference lines
for price in [50, 100, 150, 200]:
    for row_num in [1, 2]:
        fig1.add_vline(
            x=price, 
            line_dash='dash', 
            line_color=SCENARIO_COLORS[f'${price}/tCO₂'],
            opacity=0.5,
            row=row_num, col=1
        )

fig1.update_xaxes(title_text='Carbon Price ($/tCO₂)', row=2, col=1)
fig1.update_yaxes(title_text='Number of Assets', row=1, col=1)
fig1.update_yaxes(title_text='Percentage (%)', row=2, col=1)

fig1.update_layout(
    height=800,
    title_text='STRANDING CASCADE: When Do Assets Become Unprofitable?',
    showlegend=False
)

fig1.show()

# Print critical thresholds
print("Critical stranding thresholds:")

for threshold in [10, 25, 50, 75]:
    threshold_row = df_cascade[df_cascade['stranded_pct'] >= threshold].iloc[0] if len(df_cascade[df_cascade['stranded_pct'] >= threshold]) > 0 else None
    if threshold_row is not None:
        print(f"  {threshold}% of assets stranded at: ${threshold_row['carbon_price']:.0f}/tCO₂")

Critical stranding thresholds:


#### 1.2 Break-Even Price Distribution

In [24]:
# Detailed break-even histogram
fig2 = go.Figure()

fig2.add_trace(go.Histogram(
    x=breakeven_valid,
    nbinsx=60,
    marker_color='#2E86AB',
    name='Assets',
    hovertemplate='Break-even: $%{x:.0f}/tCO₂<br>Count: %{y}<extra></extra>'
))

# Add scenario lines
for price in [50, 100, 150, 200]:
    assets_below = (breakeven_valid < price).sum()
    pct_below = assets_below / len(breakeven_valid) * 100
    
    fig2.add_vline(
        x=price,
        line_dash='dash',
        line_color=SCENARIO_COLORS[f'${price}/tCO₂'],
        line_width=2,
    )

fig2.update_layout(
    title='Break-Even Carbon Price Distribution (30% Margin Threshold)',
    xaxis_title='Break-even Carbon Price ($/tCO₂)',
    yaxis_title='Number of Assets',
    height=600,
    showlegend=False
)

fig2.show()

# Summary table
print("Assets with break-even BELOW scenario price (at risk of stranding):")
for price in [50, 100, 150, 200, 250]:
    count = (breakeven_valid < price).sum()
    pct = count / len(breakeven_valid) * 100
    print(f"  ${price:>3}/tCO₂: {count:>4} assets ({pct:>5.1f}% of portfolio)")

Assets with break-even BELOW scenario price (at risk of stranding):
  $ 50/tCO₂:    0 assets (  0.0% of portfolio)
  $100/tCO₂:    2 assets (  1.1% of portfolio)
  $150/tCO₂:    2 assets (  1.1% of portfolio)
  $200/tCO₂:    3 assets (  1.7% of portfolio)
  $250/tCO₂:    5 assets (  2.8% of portfolio)


#### 1.3 Top Assets by Break-Even Price (Most Vulnerable)

In [25]:
# Lowest break-even = most vulnerable to stranding
top_vulnerable = df_financial[df_financial['break_even_carbon_price'].notna()].nsmallest(30, 'break_even_carbon_price')[[
    'source_name', 'iso3_country', 'parent_name', 'mine_type',
    'estimated_revenue_usd', 'annual_emissions_t_co2e', 'carbon_intensity',
    'break_even_carbon_price', 'carbon_cost_pct_revenue_100', 'underwater_at_100'
]].copy()

top_vulnerable.columns = ['Mine', 'Country', 'Parent', 'Type', 'Revenue', 
                         'Emissions', 'Intensity', 'Break-even Price',
                         'Carbon Cost % @$100', 'Underwater @$100']

print("Top 30 most vulnerable assets (lowest break-even prices)")
print("These assets become unprofitable at the lowest carbon prices\n")

display(top_vulnerable.style.format({
    'Revenue': '${:,.0f}',
    'Emissions': '{:,.0f}',
    'Intensity': '{:.4f}',
    'Break-even Price': '${:.0f}',
    'Carbon Cost % @$100': '{:.1f}%'
}).background_gradient(subset=['Break-even Price'], cmap='Reds'))

Top 30 most vulnerable assets (lowest break-even prices)
These assets become unprofitable at the lowest carbon prices



,Mine,Country,Parent,Type,Revenue,Emissions,Intensity,Break-even Price,Carbon Cost % @$100,Underwater @$100
212,El Salvador Mine,CHL,Codelco Corp,Both,"$54,738,392","216,433",0.3005,$76,39.5%,True
607,Sepon Mine,LAO,Chifeng Jilong Gold Mining Co Ltd,Open Pit,"$30,400,000","110,560",0.2764,$82,36.4%,True
805,Las Cruces Mine,ESP,Cobre Las Cruces SA,Open Pit,"$2,708,336","5,000",0.1403,$163,18.5%,False
66,Deflector Operation,AUS,Silver Lake Resources Ltd,Underground,"$80,973,896","105,373",0.0989,$231,13.0%,False
524,Tenke Fungurume Mine,COD,CMOC Group Ltd,Open Pit,"$1,079,139,200","1,340,404",0.0944,$242,12.4%,False
27,Eloise Mine,AUS,AIC Mines Ltd,Underground,"$43,654,400","46,469",0.0809,$282,10.6%,False
91,Palito Mine,BRA,Serabi Gold PLC,Underground,"$13,518,120","13,767",0.0774,$295,10.2%,False
50,Nova Mine,AUS,IGO Ltd,Underground,"$114,007,676","111,007",0.0740,$308,9.7%,False
662,Kainantu Mine,PNG,K92 Holdings International Ltd,Underground,"$38,480,168","34,176",0.0675,$338,8.9%,False
545,Batu Hijau Mine,IDN,PT Amman Mineral Internasional Tbk,Open Pit,"$873,495,442","681,556",0.0593,$384,7.8%,False


---
## PART 2: Profitability Impact Analysis

#### 2.1 Carbon Cost as % of Revenue Distribution

In [26]:
# Create box plot showing distribution across scenarios
impact_data = []
for scenario in [50, 100, 150, 200]:
    col = f'carbon_cost_pct_revenue_{scenario}'
    impact_data.append({
        'Scenario': f'${scenario}/tCO₂',
        'Values': df_financial[col].values
    })

fig3 = go.Figure()

for item in impact_data:
    fig3.add_trace(go.Box(
        y=item['Values'],
        name=item['Scenario'],
        marker_color=SCENARIO_COLORS[item['Scenario']],
        boxmean='sd'
    ))

# Add 30% margin line
fig3.add_hline(
    y=30,
    line_dash='dash',
    line_color='red',
    line_width=3,
    annotation_text='30% Margin Threshold (Stranding Point)',
    annotation_position='right'
)

fig3.update_layout(
    title='Carbon Cost as % of Revenue - Distribution Across Scenarios',
    yaxis_title='Carbon Cost as % of Revenue',
    xaxis_title='Scenario',
    height=600,
    showlegend=False
)

fig3.show()

# Statistics
print("Carbon Cost as percentage of Revenue - Key Statistics:")

for scenario in [50, 100, 150, 200]:
    col = f'carbon_cost_pct_revenue_{scenario}'
    data = df_financial[col]
    print(f"\n${scenario}/tCO₂:")
    print(f"  Median: {data.median():.2f}%")
    print(f"  Mean: {data.mean():.2f}%")
    print(f"  Assets > 30% (stranded): {(data > 30).sum()} ({(data > 30).sum()/len(data)*100:.1f}%)")
    print(f"  Assets > 50%: {(data > 50).sum()} ({(data > 50).sum()/len(data)*100:.1f}%)")

Carbon Cost as percentage of Revenue - Key Statistics:

$50/tCO₂:
  Median: 0.99%
  Mean: 1.21%
  Assets > 30% (stranded): 0 (0.0%)
  Assets > 50%: 0 (0.0%)

$100/tCO₂:
  Median: 1.97%
  Mean: 2.43%
  Assets > 30% (stranded): 2 (0.3%)
  Assets > 50%: 0 (0.0%)

$150/tCO₂:
  Median: 2.96%
  Mean: 3.64%
  Assets > 30% (stranded): 2 (0.3%)
  Assets > 50%: 2 (0.3%)

$200/tCO₂:
  Median: 3.95%
  Mean: 4.85%
  Assets > 30% (stranded): 3 (0.5%)
  Assets > 50%: 2 (0.3%)


#### 2.2 Scatter: Revenue vs Carbon Cost Impact

In [27]:
# Interactive scatter showing profitability impact
fig4 = px.scatter(
    df_financial,
    x='estimated_revenue_usd',
    y='carbon_cost_pct_revenue_100',
    size='annual_emissions_t_co2e',
    color='underwater_at_100',
    hover_data=['source_name', 'iso3_country', 'parent_name', 'break_even_carbon_price'],
    title='Revenue vs Carbon Cost Impact ($100/tCO₂ Scenario)',
    labels={
        'estimated_revenue_usd': 'Estimated Annual Revenue (USD)',
        'carbon_cost_pct_revenue_100': 'Carbon Cost as % of Revenue',
        'underwater_at_100': 'Unprofitable at $100/t'
    },
    color_discrete_map={True: '#C73E1D', False: '#2E86AB'},
    height=700
)

# Add stranding threshold line
fig4.add_hline(
    y=30,
    line_dash='dash',
    line_color='red',
    annotation_text='Stranding Threshold (30%)',
    annotation_position='right'
)

fig4.update_xaxes(type='log')
fig4.update_layout(legend_title_text='Asset Status')

fig4.show()

 **INTERPRETATION:**
 - **Assets ABOVE red line:** Carbon costs exceed operating margin (stranded)
 - **Assets BELOW red line:** Still profitable but at risk
 - **Bubble size:** Total emissions (larger = higher exposure)

---
## PART 3: Sector/Mine Type Breakdown

#### 3.1 Mine Type Risk Analysis

In [28]:
# Aggregate by mine type
mine_type_analysis = df_financial.groupby('mine_type').agg({
    'source_id': 'count',
    'annual_emissions_t_co2e': ['sum', 'mean'],
    'carbon_intensity': 'median',
    'break_even_carbon_price': 'median',
    'carbon_cost_pct_revenue_100': 'median',
    'underwater_at_50': 'sum',
    'underwater_at_100': 'sum',
    'underwater_at_150': 'sum',
    'underwater_at_200': 'sum'
}).reset_index()

mine_type_analysis.columns = ['Mine Type', 'Count', 'Total Emissions', 'Avg Emissions',
                              'Median Intensity', 'Median Break-even',
                              'Median Cost % @$100', 'Stranded @$50',
                              'Stranded @$100', 'Stranded @$150', 'Stranded @$200']

# Calculate stranding rates
for scenario in [50, 100, 150, 200]:
    mine_type_analysis[f'Stranding Rate @${scenario}'] = (
        mine_type_analysis[f'Stranded @${scenario}'] / mine_type_analysis['Count'] * 100
    )

# Sort by vulnerability (highest stranding rate at $100)
mine_type_analysis = mine_type_analysis.sort_values('Stranding Rate @$100', ascending=False)

print(" Mine Type Vulnerability Ranking")


display(mine_type_analysis[[
    'Mine Type', 'Count', 'Median Intensity', 'Median Break-even',
    'Stranding Rate @$50', 'Stranding Rate @$100', 
    'Stranding Rate @$150', 'Stranding Rate @$200'
]].style.format({
    'Median Intensity': '{:.4f}',
    'Median Break-even': '${:.0f}',
    'Stranding Rate @$50': '{:.1f}%',
    'Stranding Rate @$100': '{:.1f}%',
    'Stranding Rate @$150': '{:.1f}%',
    'Stranding Rate @$200': '{:.1f}%'
}).background_gradient(subset=['Stranding Rate @$100'], cmap='Reds'))

 Mine Type Vulnerability Ranking


,Mine Type,Count,Median Intensity,Median Break-even,Stranding Rate @$50,Stranding Rate @$100,Stranding Rate @$150,Stranding Rate @$200
0,Both,57,0.0124,$1839,0.0%,1.8%,1.8%,1.8%
1,Open Pit,315,0.0159,$1434,0.0%,0.3%,0.3%,0.6%
2,Underground,229,0.0122,$1869,0.0%,0.0%,0.0%,0.0%


In [29]:
# Visualization: Mine type comparison
fig5 = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Median Carbon Intensity by Mine Type', 'Stranding Rate at $100/tCO₂'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}]]
)

# Intensity plot
fig5.add_trace(
    go.Bar(
        x=mine_type_analysis['Mine Type'],
        y=mine_type_analysis['Median Intensity'],
        marker_color='#F18F01',
        name='Intensity',
        text=mine_type_analysis['Median Intensity'].round(4),
        textposition='outside'
    ),
    row=1, col=1
)

# Stranding rate plot
fig5.add_trace(
    go.Bar(
        x=mine_type_analysis['Mine Type'],
        y=mine_type_analysis['Stranding Rate @$100'],
        marker_color='#C73E1D',
        name='Stranding Rate',
        text=[f"{val:.1f}%" for val in mine_type_analysis['Stranding Rate @$100']],
        textposition='outside'
    ),
    row=1, col=2
)

fig5.update_xaxes(tickangle=-45, row=1, col=1)
fig5.update_xaxes(tickangle=-45, row=1, col=2)
fig5.update_yaxes(title_text='tCO₂/t ore', row=1, col=1)
fig5.update_yaxes(title_text='% of Assets Stranded', row=1, col=2)

fig5.update_layout(
    height=600,
    title_text='Mine Type Vulnerability Analysis',
    showlegend=False
)

fig5.show()

#### 3.2 Country-Sector Risk Matrix

In [30]:
# Top 10 countries by asset count
top_countries = df_financial['iso3_country'].value_counts().head(10).index

# Country-Mine Type cross-tabulation
country_type_matrix = df_financial[df_financial['iso3_country'].isin(top_countries)].groupby(
    ['iso3_country', 'mine_type']
).agg({
    'underwater_at_100': 'sum',
    'source_id': 'count'
}).reset_index()

country_type_matrix['stranding_rate'] = (
    country_type_matrix['underwater_at_100'] / country_type_matrix['source_id'] * 100
)

# Pivot for heatmap
heatmap_data = country_type_matrix.pivot(
    index='iso3_country',
    columns='mine_type',
    values='stranding_rate'
).fillna(0)

# Create heatmap
fig6 = px.imshow(
    heatmap_data,
    labels=dict(x='Mine Type', y='Country', color='Stranding Rate (%)'),
    x=heatmap_data.columns,
    y=heatmap_data.index,
    color_continuous_scale='Reds',
    title='Country-Mine Type Stranding Risk Matrix ($100/tCO₂)',
    height=600,
    text_auto='.1f'
)

fig6.show()


###  INTERPRETATION:
 - **Darker red = Higher stranding risk** for that country-mine type combination
 - **Helps identify which sectors are most vulnerable** in each geography

---
## PART 4: Temporal Analysis (2021-2024 Trends)

#### 4.1 Emission Trajectory Classification

In [31]:
# Aggregate monthly to annual
annual_trends = df_monthly.groupby(['source_id', 'source_name', 'iso3_country', 'year']).agg({
    'emissions_t_co2e': 'sum',
    'activity': 'sum',
    'capacity': 'mean'
}).reset_index()

# Filter to 2021-2024
annual_trends = annual_trends[annual_trends['year'].between(2021, 2024)]

# Calculate trend for each asset (linear regression)
asset_trajectories = []

for asset_id in annual_trends['source_id'].unique():
    asset_data = annual_trends[annual_trends['source_id'] == asset_id].sort_values('year')
    
    if len(asset_data) >= 3:  # Need at least 3 years
        slope, intercept, r_value, p_value, std_err = stats.linregress(
            asset_data['year'], asset_data['emissions_t_co2e']
        )
        
        # Get emissions values
        emissions_2021 = asset_data[asset_data['year']==2021]['emissions_t_co2e'].values
        emissions_2024 = asset_data[asset_data['year']==2024]['emissions_t_co2e'].values
        
        asset_trajectories.append({
            'source_id': asset_id,
            'source_name': asset_data['source_name'].iloc[0],
            'iso3_country': asset_data['iso3_country'].iloc[0],
            'trend_slope': slope,
            'trend_r2': r_value**2,
            'avg_emissions': asset_data['emissions_t_co2e'].mean(),
            'emissions_2021': emissions_2021[0] if len(emissions_2021) > 0 else np.nan,
            'emissions_2024': emissions_2024[0] if len(emissions_2024) > 0 else np.nan,
            'years_observed': len(asset_data)
        })

df_trajectories = pd.DataFrame(asset_trajectories)

# Calculate % change
df_trajectories['pct_change_2021_2024'] = (
    (df_trajectories['emissions_2024'] - df_trajectories['emissions_2021']) / 
    df_trajectories['emissions_2021'] * 100
)

# Classify trajectories
def classify_trajectory(row):
    if pd.isna(row['pct_change_2021_2024']):
        return 'Insufficient Data'
    elif row['pct_change_2021_2024'] < -10:
        return 'Improving (>10% reduction)'
    elif row['pct_change_2021_2024'] < -2:
        return 'Slightly Improving'
    elif row['pct_change_2021_2024'] <= 2:
        return 'Stable'
    elif row['pct_change_2021_2024'] <= 10:
        return 'Slightly Deteriorating'
    else:
        return 'Deteriorating (>10% increase)'

df_trajectories['trajectory_category'] = df_trajectories.apply(classify_trajectory, axis=1)

print(" Emission Trajectory Classification (2021-2024)")
print(f"\nAssets with sufficient data: {len(df_trajectories)}")
print(f"\nTrajectory Distribution:")
print(df_trajectories['trajectory_category'].value_counts())

# Merge with financial data
df_financial_with_trend = df_financial.merge(
    df_trajectories[['source_id', 'trend_slope', 'pct_change_2021_2024', 'trajectory_category']],
    on='source_id',
    how='left'
)

 Emission Trajectory Classification (2021-2024)

Assets with sufficient data: 914

Trajectory Distribution:
trajectory_category
Insufficient Data                296
Stable                           208
Deteriorating (>10% increase)    202
Improving (>10% reduction)       136
Slightly Deteriorating            41
Slightly Improving                31
Name: count, dtype: int64


#### 4.2 Trajectory vs Stranding Risk Analysis

In [32]:
# Cross-tabulation: Trajectory vs Underwater status
trajectory_risk = df_financial_with_trend.groupby('trajectory_category').agg({
    'source_id': 'count',
    'underwater_at_100': 'sum',
    'break_even_carbon_price': 'median',
    'carbon_cost_pct_revenue_100': 'median'
}).reset_index()

trajectory_risk.columns = ['Trajectory', 'Count', 'Stranded @$100', 
                          'Median Break-even', 'Median Cost %']
trajectory_risk['Stranding Rate'] = trajectory_risk['Stranded @$100'] / trajectory_risk['Count'] * 100

# Sort by risk
trajectory_risk = trajectory_risk.sort_values('Stranding Rate', ascending=False)

print("Trajectory vs Stranding risk")


display(trajectory_risk.style.format({
    'Median Break-even': '${:.0f}',
    'Median Cost %': '{:.1f}%',
    'Stranding Rate': '{:.1f}%'
}).background_gradient(subset=['Stranding Rate'], cmap='RdYlGn_r'))


Trajectory vs Stranding risk


,Trajectory,Count,Stranded @$100,Median Break-even,Median Cost %,Stranding Rate
1,Improving (>10% reduction),119,1,$1839,1.6%,0.8%
0,Deteriorating (>10% increase),202,1,$1520,2.0%,0.5%
2,Slightly Deteriorating,41,0,$1434,2.1%,0.0%
3,Slightly Improving,31,0,$1781,1.7%,0.0%
4,Stable,208,0,$1490,2.0%,0.0%


#### KEY INSIGHT:
 - Assets with **DETERIORATING emissions** + **HIGH current exposure** = *Highest risk*
 - Assets with **IMPROVING emissions** may still be at risk if starting from a high baseline

#### 4.3 Worst Offenders - Deteriorating + High Exposure

In [33]:
# Identify worst offenders: Deteriorating trend + High exposure
worst_offenders = df_financial_with_trend[
    (df_financial_with_trend['pct_change_2021_2024'] > 10) &  # >10% increase
    (df_financial_with_trend['carbon_cost_pct_revenue_100'] > 20)  # >20% of revenue
].sort_values('carbon_cost_pct_revenue_100', ascending=False)[
    ['source_name', 'iso3_country', 'parent_name', 'mine_type',
     'pct_change_2021_2024', 'carbon_cost_pct_revenue_100',
     'break_even_carbon_price', 'underwater_at_100']
].head(20)

worst_offenders.columns = ['Mine', 'Country', 'Parent', 'Type',
                          'Emissions Change 2021-2024 (%)', 'Carbon Cost % @$100',
                          'Break-even Price', 'Stranded @$100']

print("Highest risk: Deteriorating Emissions + High Current Exposure")

display(worst_offenders.style.format({
    'Emissions Change 2021-2024 (%)': '{:+.1f}%',
    'Carbon Cost % @$100': '{:.1f}%',
    'Break-even Price': '${:.0f}'
}).background_gradient(subset=['Emissions Change 2021-2024 (%)', 'Carbon Cost % @$100'], cmap='Reds'))

Highest risk: Deteriorating Emissions + High Current Exposure


,Mine,Country,Parent,Type,Emissions Change 2021-2024 (%),Carbon Cost % @$100,Break-even Price,Stranded @$100
101,El Salvador Mine,CHL,Codelco Corp,Both,+102.4%,39.5%,$76,True


#### 4.4 Visualization: Emission Trajectories for Top 20 Exposed Assets

In [34]:
# Get top 20 by carbon cost
top20_ids = df_financial.nlargest(20, 'carbon_cost_usd_100')['source_id'].tolist()

# Get their annual trends
top20_trends = annual_trends[annual_trends['source_id'].isin(top20_ids)]

# Create line plot
fig7 = px.line(
    top20_trends,
    x='year',
    y='emissions_t_co2e',
    color='source_name',
    markers=True,
    title='Emission Trajectories: Top 20 Most Exposed Assets (2021-2024)',
    labels={'emissions_t_co2e': 'Annual Emissions (tCO₂)', 'year': 'Year'},
    height=700
)

fig7.update_traces(line=dict(width=2), marker=dict(size=8))
fig7.update_layout(legend_title_text='Mine')

fig7.show()

print(" Shows whether high-exposure assets are improving or getting worse over time")

 Shows whether high-exposure assets are improving or getting worse over time


#### 4.5 Global Trend Summary

In [35]:
# Global annual totals
global_annual = annual_trends.groupby('year')['emissions_t_co2e'].sum().reset_index()
global_annual.columns = ['Year', 'Total Emissions']

# Calculate year-over-year change
global_annual['YoY Change (%)'] = global_annual['Total Emissions'].pct_change() * 100

print(" Global Copper Mining Emissions Trend")
display(global_annual.style.format({
    'Total Emissions': '{:,.0f} tCO₂',
    'YoY Change (%)': '{:+.2f}%'
}).background_gradient(subset=['YoY Change (%)'], cmap='RdYlGn_r'))

# CAGR calculation
if len(global_annual) > 1:
    years = global_annual['Year'].max() - global_annual['Year'].min()
    cagr = ((global_annual['Total Emissions'].iloc[-1] / global_annual['Total Emissions'].iloc[0]) ** (1/years) - 1) * 100
    print(f" Compound Annual Growth Rate (CAGR): {cagr:+.2f}% per year")
    print(f"{'Increasing emissions trend!' if cagr > 0 else 'Decreasing emissions trend!'}")

 Global Copper Mining Emissions Trend


,Year,Total Emissions,YoY Change (%)
0,2021,"92,425,148 tCO₂",+nan%
1,2022,"95,168,205 tCO₂",+2.97%
2,2023,"103,603,874 tCO₂",+8.86%
3,2024,"95,204,706 tCO₂",-8.11%


 Compound Annual Growth Rate (CAGR): +0.99% per year
Increasing emissions trend!


---
## PART 5: Export Enhanced Datasets for Dashboard Integration

In [36]:
# Export comprehensive financial dataset
export_columns = [
    'source_id', 'source_name', 'iso3_country', 'parent_name', 'mine_type',
    'lat', 'lon', 'mine_status',
    'annual_emissions_t_co2e', 'annual_production_t_ore', 'carbon_intensity',
    'estimated_revenue_usd', 'break_even_carbon_price',
    'carbon_cost_pct_revenue_50', 'carbon_cost_pct_revenue_100',
    'carbon_cost_pct_revenue_150', 'carbon_cost_pct_revenue_200',
    'underwater_at_50', 'underwater_at_100', 'underwater_at_150', 'underwater_at_200',
    'trend_slope', 'pct_change_2021_2024', 'trajectory_category'
]

df_export = df_financial_with_trend[export_columns].copy()
df_export.to_csv('../datasets/output_data/financial/financial_analysis_complete.csv', index=False)

# Export summary tables
mine_type_analysis.to_csv('../datasets/output_data/financial/mine_type_vulnerability.csv', index=False)
trajectory_risk.to_csv('../datasets/output_data/financial/trajectory_risk_analysis.csv', index=False)
df_cascade.to_csv('../datasets/output_data/financial/stranding_cascade_data.csv', index=False)

# Export priority lists
top_vulnerable.to_csv('../datasets/output_data/financial/top_vulnerable_assets.csv', index=False)
if len(worst_offenders) > 0:
    worst_offenders.to_csv('../datasets/output_data/financial/deteriorating_high_exposure_assets.csv', index=False)

## Conclusion
 
In this analysis, we have examined the global trend in copper mining emissions, assessed site-level financial vulnerability under various carbon pricing scenarios, and identified assets at highest risk of stranding. The integration of financial, operational, and emissions data enables stakeholders to better understand both the direct and systemic risks posed by the energy transition and increasing climate policies. The exported datasets and summaries provided here can support further decision-making, strategic planning, and future dashboard integration. Continued monitoring and enhanced disclosure will be essential as the sector and regulators advance toward net-zero goals.
